# Key Concepts

## Jax 배열 (`jax.Array`)

`jax.Array`는 jax의 기본 배열입니다. `numpy.ndarray`와 거의 유사하지만, 몇가지 중요한 차이점이 있습니다.

### 배열 생성

비록 `jax.Array`는 jax의 기본 배열이지만, 보통 저희가 실제 구현할 때는 `jax.Array` 를 직접 호출하는 대신 jax의 API 함수인 `jax.numpy`를 호출해서 사용하게 됩니다. `jax.numpy` 는 `jax.numpy.zeros()`, `jax.numpy.linspace()`, `jax.numpy.arange()` 와 같이 NumPy 스타일의 배열생성 함수를 제공합니다.

In [1]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
isinstance(x, jax.Array)

True

다만 파이썬 어노테이션을 사용할 때는 `jax.Array`를 사용하시면 됩니다:

In [5]:
# 공식 도큐먼트에는 없는 코드입니다.
@jax.jit
def apply_matrix(x: jax.Array) -> jax.Array:
    return x ** 2

apply_matrix(jnp.arange(5))

Array([ 0,  1,  4,  9, 16], dtype=int32)

### devices와 sharding

Jax 배열이 가지고 있는 속성인 `devices` 는 배열을 어떤 연산 장치(GPU, CPU 등)에 할당되어 있는지를 나타냅니다. 기본적으로 cpu에 할당되어 있을 겁니다.

In [6]:
x.devices()

{CpuDevice(id=0)}

일반적으로 jax 배열은 여러개의 장치에 `sharding`되어 있습니다. `sharding`이란 DB에서 나온 용어이며, 여기서는 여러 장치에 복사되어 있다는 의미로 쓰였습니다. `sharding` 속성을 통해 샤딩 상태를 확인하실 수 있습니다.

In [8]:
x.sharding

SingleDeviceSharding(device=CpuDevice(id=0))

비록 예제에서는 single device 에만 저장되어 있지만, 일반적으로 jax 배열은 여러 연산 장치 뿐만 아니라 여러 호스트(컴퓨터)에도 `sharding`될 수 있습니다.

### 변환함수(Transformations)

다음은 변환함수 중 일부입니다:
- `jax.jit()`: 함수를 컴파일합니다.
- `jax.vmap()`: 함수를 벡터화합니다.
- `jax.grad()`: 함수의 도함수를 구합니다.

변환함수는 함수를 인자로 받아 변한된 함수를 리턴합니다. 예로 `jax.jit()`를 사용하여 제곱 함수를 컴파일해 보겠습니다:

In [2]:
def square(x):
    return jnp.pow(x, 2)

square_jit = jax.jit(square)
print(square_jit(2.0))

4.0


아래와 같이 파이썬 데코레이터를 사용할 수도 있습니다:

In [10]:
@jax.jit
def square(x):
    return jnp.pow(x, 2)

### 트레이싱(Tracing)

모든 변환함수의 마법같은 작동 뒤에는 `Tracer` 가 있습니다. 함수를 변환할 때는 인풋에 대한 아웃풋 값(함수의 결과)이 중요한 것이 아니라, 함수가 어떻게 인풋 값을 수정하고 변환하는지(함수의 과정)가 중요합니다. 그래야 함수의 역할을 왜곡하지 않고 변환할 수 있기 때문입니다. 이때 jax array 의 자리애 Tracer가 대신 대체되어 들어가게 되고, 이는 jax가 함수의 역할을 잘 이해할 수 있도록 돕습니다. 

In [4]:
@jax.jit
def f(x):
  print(x)
  return x + 1

x = jnp.arange(5)
result = f(x)

Traced<ShapedArray(int32[5])>with<DynamicJaxprTrace(level=1/0)>


일반적인 사용은 아니지만, `Tracer`의 개념을 확인 하기 위한 방법으로, `jit`을 통해 컴파일되는 함수 중간에 `print()` 함수를 통해 x의 값을 중간에 출력해 보았습니다. 우리가 기대한 `[0, 1, 2, 3, 4]` 의 `jax.array`가 아닌, `Traced` 타입의 값이 반환되었습니다. `Traced`값에는 배열의 값은 없지만, `shape`, `dtype`과 같은 인풋 배열의 중요 정보들을 포함하고 있는데, 이 정보들을 통해 jax는 함수가 정확히 어떠한 연산을 수행하는지 알 수 있게 됩니다. 그 덕에 `jit()`, `vmap()`, `grad()`와 같은 변환함수들이 함수의 연산을 그대로 보존하며 원하는 동작을 수행할 수 있게 됩니다.

### Jaxprs

Jax는 일렬의 연산과정을 표현하는 자신만의 방법이 있는데, 이를 `jaxprs`라고 합니다(읽을 때는 `jax expression`으로 읽으면 됩니다).  

저희가 만들었던 `selu`함수를 다시 살펴보겠습니다:

In [5]:
def selu(x, alpha=1.67, lambda_=1.05):
  return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

`jax.make_jaxpr()` 함수를 통해 `selu` 함수를 `jaxpr` 형태로 나타낼 수 있습니다:

In [6]:
x = jnp.arange(5.0)
jax.make_jaxpr(selu)(x)

{ lambda ; a:f32[5]. let
    b:bool[5] = gt a 0.0
    c:f32[5] = exp a
    d:f32[5] = mul 1.6699999570846558 c
    e:f32[5] = sub d 1.6699999570846558
    f:f32[5] = pjit[
      name=_where
      jaxpr={ lambda ; g:bool[5] h:f32[5] i:f32[5]. let
          j:f32[5] = select_n g i h
        in (j,) }
    ] b a e
    k:f32[5] = mul 1.0499999523162842 f
  in (k,) }

파이썬 코드와 비교했을 때, `selu`의 연산 과정을 정확히 표현해내는 것을 볼 수 있습니다.

만약 `selu` 함수가 익숙하지 않으시다면, 익숙한 형태의 함수를 만든 후 `jax.make_jaxpr()` 를 시도해보시는 것도 추전드립니다.

### Pytrees

근본적으로 Jax는 배열을 통해 연산이 이루어지지만, 많은 Real-world 문제들에서는 순수하게 배열 대 배열로 연산이 이루어지지만은 않습니다. 예를 들어, 딥러닝을 이루고 있는 신경망의 경우 여러개의 행렬이 순차적으로 행렬곱이 시행되는데, 신경망 연산을 할 때마다 배열 단위로 뜯어서 Jax 연산을 하고 싶지는 않으실 겁니다.

In [2]:
# (nested) list of parameters
params = [1, 2, (jnp.arange(3), jnp.ones(2))]

print(jax.tree.structure(params))
print(jax.tree.leaves(params))

PyTreeDef([*, *, (*, *)])
[1, 2, Array([0, 1, 2], dtype=int32), Array([1., 1.], dtype=float32)]


In [3]:
# Dictionary of parameters
params = {'n': 5, 'W': jnp.ones((2, 2)), 'b': jnp.zeros(2)}

print(jax.tree.structure(params))
print(jax.tree.leaves(params))

PyTreeDef({'W': *, 'b': *, 'n': *})
[Array([[1., 1.],
       [1., 1.]], dtype=float32), Array([0., 0.], dtype=float32), 5]


In [4]:
# Named tuple of parameters
from typing import NamedTuple

class Params(NamedTuple):
  a: int
  b: float

params = Params(1, 5.0)
print(jax.tree.structure(params))
print(jax.tree.leaves(params))

PyTreeDef(CustomNode(namedtuple[Params], [*, *]))
[1, 5.0]
